In [ ]:
import torch

if torch.cuda.is_available():    
   
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))
    !nvidia-smi

else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: Tesla T4
Thu Mar 16 00:00:39 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   58C    P0    26W /  70W |      3MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                            

In [ ]:
import pandas as pd
import numpy as np
import re
from sklearn.model_selection import train_test_split

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
# https://blog.tensorflow.org/2019/11/hugging-face-state-of-art-natural.html
!pip install transformers
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 94.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 KB 20.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 71.1 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.0/469.0 KB 37.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 KB 16.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 KB 30.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 57.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 KB 15.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 KB 22.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 KB 14.9 MB/s eta 0:00:0

In [ ]:
pip install wandb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 74.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 KB 15.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 KB 21.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 KB 4.7 MB/s eta 0:00:00
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8807 sha256=e0e748e707bb820ecbd90e3c3e75216668a39eb84cbcee64713c8d5493051816
  Stored in directory: /root/.cache/pip/wheels/b7/0a/67/ada2a22079218c75a88361c0782855cc72aebc4d18d0289d05
Successfully built pathtools


In [ ]:
import numpy as np
import pandas as pd

import transformers
from datasets import Dataset,load_dataset,load_from_disk, DatasetDict
from transformers import AutoTokenizer, AutoModelForSequenceClassification

from transformers import BertForSequenceClassification, BertTokenizer

In [ ]:
ar_df = pd.read_csv("./datasets/preprocessed/all_ar.csv", index_col=[0]).sample(frac=1).reset_index(drop=True)


In [ ]:
ar_df.head(5)

,tweet,label
0,الهيئة ايران راح تغطي علي الصين في الاصابات بف...,fake
1,الخارجية الصينية: نعارض بشدة القيود التمييزية ...,real
2,الخارجية الإيرانية: لسنا جزءا من الحرب بأوكران...,real
3,وزير الدفاع الأمريكي: أوروبا تواجه أسوأ أزمة أ...,real
4,أنباء عن وفاة طفل فيروس_كورونا في مشفى المجت...,fake


In [ ]:
ar_train_x, ar_test_x, ar_train_y, ar_test_y = train_test_split(ar_df["tweet"], ar_df["label"], test_size=0.2, random_state=2)

ar_train_x, ar_val_x, ar_train_y, ar_val_y = train_test_split (ar_train_x, ar_train_y, test_size=0.13, random_state=2)

ar_train_df = pd.DataFrame(list(zip(ar_train_x, ar_train_y)),
               columns =['tweet', 'label'])

ar_val_df = pd.DataFrame(list(zip(ar_val_x, ar_val_y)),
               columns =['tweet', 'label'])


ar_test_df = pd.DataFrame(list(zip(ar_test_x, ar_test_y)),
               columns =['tweet', 'label'])

print(ar_train_df.shape)
print(ar_val_df.shape)
print(ar_test_df.shape)

(945, 2)
(142, 2)
(272, 2)


In [ ]:
print(ar_train_x.head())
print(ar_train_y.head())

1289    وزير الصحة قال: المصابة بال  كورونا هي بحالة ج...
317     هذولا كيف يأكلونه المشكله ان فيروس كورونا يجي ...
505     إيقاف شحنة مساعدات إنسانية من الإمارات إلى أهل...
128     رويترز: قوات  سوريا الديمقراطية تعلن تعليق الت...
284     هوبا اللهم لا شماته   لاحولا ولا قوة إلا بالله...
Name: tweet, dtype: object
1289      fake
317       fake
505     satire
128       real
284       fake
Name: label, dtype: object


In [ ]:
ar_train_df.head()

,tweet,label
0,وزير الصحة قال: المصابة بال كورونا هي بحالة ج...,fake
1,هذولا كيف يأكلونه المشكله ان فيروس كورونا يجي ...,fake
2,إيقاف شحنة مساعدات إنسانية من الإمارات إلى أهل...,satire
3,رويترز: قوات سوريا الديمقراطية تعلن تعليق الت...,real
4,هوبا اللهم لا شماته لاحولا ولا قوة إلا بالله...,fake


In [ ]:
en_df = pd.read_csv("./datasets/preprocessed/all_en.csv", index_col=[0]).sample(frac=1).reset_index(drop=True)

In [ ]:
en_df.head(5)

,tweet,label
0,fbi releases list of 10 weirdest people who ar...,satire
1,"soon 67\nnever had flu vaccine, never had flu\...",fake
2,‘sesame street’ introduces first enigmatic mup...,satire
3,subscribe to our letter for the latest from t...,satire
4,",""pope to un: use covid crisis to come out bet...",fake


In [ ]:
all_df = pd.concat([ar_train_df, en_df], axis=0).sample(frac=1).reset_index(drop=True)

In [ ]:
all_df.head(10)

,tweet,label
0,activist asks court to reduce sentence from tr...,satire
1,"donald trump diagnosed with adtd, or attention...",fake
2,فوز ماكرون بولاية ثانية بعد إقناعه الناخبين بب...,satire
3,this precedented humanitarian gesture will dra...,satire
4,الدون كريستيانو رونالدو يقرر تحويل جميع فنادقه...,fake
5,a story by italian tv rai proves coronavirus ...,fake
6,ukrainian president zelenskyy calls russia's s...,real
7,البحرين - حصريا في عيد الحب، رسالة بقلم جلالة ...,satire
8,"we’re in the middle of a global pandemic, and ...",fake
9,"for more exemplary journalism, subscribe to ou...",satire


In [ ]:
all_df.shape

(3138, 2)

In [ ]:
##save to csv files

ar_train_df.to_csv("datasets/preprocessed/ar_train.csv", index=False)
ar_val_df.to_csv("datasets/preprocessed/ar_val.csv", index=False)
ar_test_df.to_csv("datasets/preprocessed/ar_test.csv", index=False)

all_df.to_csv("datasets/preprocessed/all.csv", index=False)

# Load Datasets to Transformer

In [ ]:
ar_train_df  = pd.read_csv("./datasets/preprocessed/ar_train.csv")
ar_val_df = pd.read_csv("./datasets/preprocessed/ar_val.csv")
ar_test_df  = pd.read_csv("./datasets/preprocessed/ar_test.csv")

all_df = pd.read_csv("./datasets/preprocessed/all.csv")

In [ ]:
all_df.head()

,tweet,label
0,activist asks court to reduce sentence from tr...,satire
1,"donald trump diagnosed with adtd, or attention...",fake
2,فوز ماكرون بولاية ثانية بعد إقناعه الناخبين بب...,satire
3,this precedented humanitarian gesture will dra...,satire
4,الدون كريستيانو رونالدو يقرر تحويل جميع فنادقه...,fake


In [ ]:
dataset_ar_train =  Dataset.from_pandas(ar_train_df,preserve_index=False)
dataset_ar_val =  Dataset.from_pandas(ar_val_df,preserve_index=False)
dataset_ar_test =  Dataset.from_pandas(ar_test_df,preserve_index=False)

dataset_ar = DatasetDict({"train": dataset_ar_train,
                       "val": dataset_ar_val,
                       "test": dataset_ar_test })

dataset_ar

DatasetDict({
    train: Dataset({
        features: ['tweet', 'label'],
        num_rows: 945
    })
    val: Dataset({
        features: ['tweet', 'label'],
        num_rows: 142
    })
    test: Dataset({
        features: ['tweet', 'label'],
        num_rows: 272
    })
})

In [ ]:
dataset_all_train = Dataset.from_pandas(all_df,preserve_index=False)

dataset_all = DatasetDict({"train": dataset_all_train,
                       "val": dataset_ar_val,
                       "test": dataset_ar_test })

dataset_all

DatasetDict({
    train: Dataset({
        features: ['tweet', 'label'],
        num_rows: 3138
    })
    val: Dataset({
        features: ['tweet', 'label'],
        num_rows: 142
    })
    test: Dataset({
        features: ['tweet', 'label'],
        num_rows: 272
    })
})

In [ ]:
dataset_all["train"]["tweet"][0:5]

['activist asks court to reduce sentence from travel ban to execution',
 'donald trump diagnosed with adtd, or attention deficit tweet disorder    donaldtrump  covid19  pandemic',
 'فوز ماكرون بولاية ثانية بعد إقناعه الناخبين ببرنامجه القائم على أنه ليس لوبين',
 'this precedented humanitarian gesture will drastically change the lives of millions while sparing their governments the trouble of collecting large amounts of  tax from multinational corporations.',
 'الدون كريستيانو رونالدو يقرر تحويل جميع فنادقه إلى مستشفيات للحجر الصحى ويتكفل برواتب جميع العاملين فيها الله عليك ما أروعك الإنسانية لا دين ولا جنسية لها يا سادة  الكورونا']

Loading models and creating tokenized datasets

Dataset class with tokenizer 

In [ ]:
from transformers.data.processors.utils import InputFeatures

class ClassificationDataset(torch.utils.data.Dataset):
    def __init__(self, tweet, label, model_name, max_len, label_map):
      super(ClassificationDataset).__init__()
      """
      Args:
      tweet (List[str]): List of tweets
      label (List[str]): List of labels
      tokenizer_name (str): The tokenizer name (same as model_name).
      max_len (int): Maximum sentence length
      label_map (Dict[str,int]): A dictionary that maps the class labels to integer
      """
      self.tweet = tweet
      self.label = label
      self.tokenizer_name = model_name
      self.tokenizer = AutoTokenizer.from_pretrained(model_name)
      self.max_len = max_len
      self.label_map = label_map
      

    def __len__(self):
      return len(self.tweet)

    def __getitem__(self,item):
      tweet = str(self.tweet[item])
      tweet = " ".join(tweet.split())
        
      inputs = self.tokenizer(
          tweet,
          max_length=self.max_len,
          padding='max_length',
          truncation=True
      )      
      return InputFeatures(**inputs,label=self.label_map[self.label[item]])

In [ ]:
label_map = {"fake": 0, "real":1 , "satire":2}
print(label_map)
max_len = 128

#### Arabic with 
train_ar_dataset = ClassificationDataset(
    dataset_ar["train"]['tweet'],
    dataset_ar["train"]['label'],
    model_name = "aubmindlab/bert-base-arabertv2",
    max_len = max_len,
    label_map = label_map,
  )

val_ar_dataset = ClassificationDataset(
    dataset_ar["val"]['tweet'],
    dataset_ar["val"]['label'],
    model_name = "aubmindlab/bert-base-arabertv2",
    max_len = max_len,
    label_map = label_map,
  )


test_ar_dataset = ClassificationDataset(
    dataset_ar["test"]['tweet'],
    dataset_ar["test"]['label'],
    model_name = "aubmindlab/bert-base-arabertv2",
    max_len = max_len,
    label_map = label_map,
  )


### All with XLM-ROBERTA ###

train_all_dataset = ClassificationDataset(
    dataset_all["train"]['tweet'],
    dataset_all["train"]['label'],
    model_name = "xlm-roberta-base",
    max_len = max_len,
    label_map = label_map,
  )

val_all_dataset = ClassificationDataset(
    dataset_all["val"]['tweet'],
    dataset_all["val"]['label'],
    model_name = "xlm-roberta-base",
    max_len = max_len,
    label_map = label_map,
  )

test_all_dataset = ClassificationDataset(
    dataset_all["test"]['tweet'],
    dataset_all["test"]['label'],
    model_name = "xlm-roberta-base",
    max_len = max_len,
    label_map = label_map,
  )



## Arabic with XLM-Roberta

train_ar_dataset_xlm_roberta = ClassificationDataset(
    dataset_ar["train"]['tweet'],
    dataset_ar["train"]['label'],
    model_name = "xlm-roberta-base",
    max_len = max_len,
    label_map = label_map,
  )

val_ar_dataset_xlm_roberta = ClassificationDataset(
    dataset_ar["val"]['tweet'],
    dataset_ar["val"]['label'],
    model_name = "xlm-roberta-base",
    max_len = max_len,
    label_map = label_map,
  )


test_ar_dataset_xlm_roberta = ClassificationDataset(
    dataset_ar["test"]['tweet'],
    dataset_ar["test"]['label'],
    model_name = "xlm-roberta-base",
    max_len = max_len,
    label_map = label_map,
  )

{'fake': 0, 'real': 1, 'satire': 2}


In [ ]:
print(next(iter(test_ar_dataset_xlm_roberta)))

InputFeatures(input_ids=[0, 188072, 139837, 53882, 47795, 4995, 18185, 230, 73752, 18109, 94665, 13308, 240, 40792, 71240, 93944, 3234, 106849, 240, 126046, 176, 65, 125963, 13767, 176, 50, 3426, 36374, 188072, 53882, 5202, 9831, 7024, 3247, 3108, 746, 97241, 4472, 125285, 17941, 648, 128094, 767, 180314, 862, 156978, 50, 153990, 6225, 4142, 6, 135154, 3047, 906, 36184, 3239, 1101, 74415, 396, 138092, 250, 1761, 746, 78699, 6, 175522, 1333, 5393, 65, 139512, 50982, 9831, 7024, 3247, 5, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], attention_mask=[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

# Loading Models

AraBERT

In [ ]:
arabert_model = AutoModelForSequenceClassification.from_pretrained("aubmindlab/bert-base-arabertv2", return_dict=True, num_labels=len(label_map)) # , num_labels=3)

Some weights of the model checkpoint at aubmindlab/bert-base-arabertv2 were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were

XLM-RobeRta, we are loading two instances of the same model for two experiments, one for english and arabic dataset, the other one is for just the arabic dataset

In [ ]:
xlm_roberta_all_model = AutoModelForSequenceClassification.from_pretrained("xlm-roberta-base", return_dict=True, num_labels=len(label_map)) # , num_labels=3)

Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaForSequenceClassification: ['lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'roberta.pooler.dense.weight', 'lm_head.decoder.weight', 'lm_head.bias', 'roberta.pooler.dense.bias', 'lm_head.dense.weight', 'lm_head.dense.bias']
- This IS expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.out_p

In [ ]:
xlm_roberta_ar_model = AutoModelForSequenceClassification.from_pretrained("xlm-roberta-base", return_dict=True, num_labels=len(label_map)) # , num_labels=3)

Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaForSequenceClassification: ['lm_head.dense.weight', 'lm_head.decoder.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'roberta.pooler.dense.bias', 'roberta.pooler.dense.weight', 'lm_head.bias']
- This IS expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.out_p

define evaluation metric

In [ ]:
from sklearn.metrics import f1_score, precision_score, accuracy_score, recall_score

def compute_metrics(p): 
  preds = np.argmax(p.predictions, axis=1)
  assert len(preds) == len(p.label_ids)
  #print(classification_report(p.label_ids,preds))
  #print(confusion_matrix(p.label_ids,preds))
  macro_f1 = f1_score(p.label_ids,preds,average='macro')
  macro_precision = precision_score(p.label_ids,preds,average='macro')
  macro_recall = recall_score(p.label_ids,preds,average='macro')
  acc = accuracy_score(p.label_ids,preds)
  return {       
      'macro_f1' : macro_f1,
      'accuracy': acc,
      'precision': macro_precision,
      'recall': macro_recall,
  }

In [ ]:
import wandb

wandb.login()

<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [ ]:
%env WANDB_LOG_MODEL=true

env: WANDB_LOG_MODEL=true


In [ ]:
from transformers import TrainingArguments, Trainer
'''
training_args_arabert = TrainingArguments( 
    output_dir= "./train-arabert",    
    adam_epsilon = 1e-8,
    learning_rate = 2e-5,
    per_device_train_batch_size = 16,
    per_device_eval_batch_size = 16,
    gradient_accumulation_steps = 2, # use this to scale batch size without needing more memory
    num_train_epochs= 4,
    
    evaluation_strategy = 'epoch',
    save_strategy = 'epoch',
    load_best_model_at_end = True, # this allows to automatically get the best model at the end based on whatever metric we want
    metric_for_best_model = 'macro_f1',
  )

  ''';

In [ ]:
from transformers import TrainingArguments, Trainer

training_args_arabert = TrainingArguments( 
    output_dir= "./train-arabert",    
    adam_epsilon = 1e-8,
    learning_rate = 2e-5,
    per_device_train_batch_size = 16,
    per_device_eval_batch_size = 16,
    gradient_accumulation_steps = 2, # use this to scale batch size without needing more memory
    num_train_epochs= 5,
    logging_strategy = 'steps',
    logging_steps = 30,
    evaluation_strategy = 'steps',
    eval_steps = 30,
    save_strategy = 'steps',
    save_steps = 30,
    load_best_model_at_end = True, # this allows to automatically get the best model at the end based on whatever metric we want
    metric_for_best_model = 'macro_f1',
    report_to = 'wandb',
    run_name = 'arabert'
  )

In [ ]:
training_args_xlm_roberta_all = TrainingArguments( 
    output_dir= "./train-xlm-robera-all",    
    adam_epsilon = 1e-8,
    learning_rate = 2e-5,
    per_device_train_batch_size = 16,
    per_device_eval_batch_size = 16,
    gradient_accumulation_steps = 2, # use this to scale batch size without needing more memory
    num_train_epochs= 5,
    logging_strategy = 'steps',
    logging_steps = 30,
    evaluation_strategy = 'steps',
    eval_steps = 30,
    save_strategy = 'steps',
    save_steps = 30,
    load_best_model_at_end = True, # this allows to automatically get the best model at the end based on whatever metric we want
    metric_for_best_model = 'macro_f1',
    report_to = 'wandb',
    run_name = 'xlm_roberta_all'
  )

In [ ]:
training_args_xlm_roberta_ar = TrainingArguments( 
    output_dir= "./train-xlm-robera-ar",    
    adam_epsilon = 1e-8,
    learning_rate = 2e-5,
    per_device_train_batch_size = 16,
    per_device_eval_batch_size = 16,
    gradient_accumulation_steps = 2, # use this to scale batch size without needing more memory
    num_train_epochs= 5,
    logging_strategy = 'steps',
    logging_steps = 30,
    evaluation_strategy = 'steps',
    eval_steps = 30,
    save_strategy = 'steps',
    save_steps = 30,
    load_best_model_at_end = True, # this allows to automatically get the best model at the end based on whatever metric we want
    metric_for_best_model = 'macro_f1',
    report_to = 'wandb',
    run_name = 'xlm_roberta_ar'
  )

Create Trainer Instance for AraBERT







Dataset = Arabic Dataset

In [ ]:
trainer_arabert = Trainer(
    model= arabert_model,
    args = training_args_arabert,
    train_dataset = train_ar_dataset,
    eval_dataset = val_ar_dataset,
    compute_metrics=compute_metrics
)

Create Trainer Instance for xlm-roberta 1

Dataset = Arabic + English





In [ ]:
trainer_xlm_robeta_all = Trainer(
    model= xlm_roberta_all_model,
    args = training_args_xlm_roberta_all,
    train_dataset = train_all_dataset,
    eval_dataset = val_all_dataset,
    compute_metrics=compute_metrics
)

Create Trainer Instance for xlm-roberta 2

Dataset = Arabic






In [ ]:
trainer_xlm_robeta_ar = Trainer(
    model= xlm_roberta_ar_model,
    args = training_args_xlm_roberta_ar,
    train_dataset = train_ar_dataset_xlm_roberta,
    eval_dataset = val_ar_dataset_xlm_roberta,
    compute_metrics=compute_metrics
)

Train Models

AraBERT on Arabic Dataset

In [ ]:
trainer_arabert.train()

/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


wandb: Currently logged in as: jehadoumer. Use `wandb login --relogin` to force relogin


Step,Training Loss,Validation Loss,Macro F1,Accuracy,Precision,Recall
30,0.759500,0.412984,0.860805,0.873239,0.872048,0.865497
60,0.282100,0.153576,0.969554,0.971831,0.969928,0.970343
90,0.140200,0.097632,0.977426,0.978873,0.977055,0.978279
120,0.079200,0.083608,0.977534,0.978873,0.976744,0.978464
150,0.052900,0.073295,0.977426,0.978873,0.977055,0.978279


TrainOutput(global_step=150, training_loss=0.26280385891596475, metrics={'train_runtime': 245.2941, 'train_samples_per_second': 19.263, 'train_steps_per_second': 0.612, 'total_flos': 310802724691200.0, 'train_loss': 0.26280385891596475, 'epoch': 5.0})

In [ ]:
arabert_model.save_pretrained("./model_arabert")

NameError: ignored

In [ ]:
!wandb login --relogin

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Train xlm roberta on the the dataset that contains english and arabic tweets

In [ ]:
trainer_xlm_robeta_all.train()

/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


wandb: Currently logged in as: jehadoumer. Use `wandb login --relogin` to force relogin


Step,Training Loss,Validation Loss,Macro F1,Accuracy,Precision,Recall
30,1.058400,0.860777,0.431082,0.570423,0.366442,0.525898
60,0.785200,0.534144,0.730466,0.781690,0.792431,0.758330
90,0.470900,0.327861,0.881801,0.894366,0.897449,0.883226
120,0.324600,0.351323,0.855367,0.873239,0.879894,0.859416
150,0.285200,0.151864,0.924376,0.929577,0.934505,0.922723
180,0.244700,0.099562,0.961818,0.964789,0.961518,0.962591
210,0.206700,0.121575,0.962731,0.964789,0.962001,0.964495
240,0.157800,0.087584,0.970316,0.971831,0.971004,0.970343
270,0.130700,0.233897,0.937773,0.943662,0.944074,0.938596
300,0.143800,0.182170,0.930983,0.936620,0.935315,0.932749


/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Step,Training Loss,Validation Loss,Macro F1,Accuracy,Precision,Recall
30,1.058400,0.860777,0.431082,0.570423,0.366442,0.525898
60,0.785200,0.534144,0.730466,0.781690,0.792431,0.758330
90,0.470900,0.327861,0.881801,0.894366,0.897449,0.883226
120,0.324600,0.351323,0.855367,0.873239,0.879894,0.859416
150,0.285200,0.151864,0.924376,0.929577,0.934505,0.922723
180,0.244700,0.099562,0.961818,0.964789,0.961518,0.962591
210,0.206700,0.121575,0.962731,0.964789,0.962001,0.964495
240,0.157800,0.087584,0.970316,0.971831,0.971004,0.970343
270,0.130700,0.233897,0.937773,0.943662,0.944074,0.938596
300,0.143800,0.182170,0.930983,0.936620,0.935315,0.932749


TrainOutput(global_step=490, training_loss=0.2612408071756363, metrics={'train_runtime': 766.388, 'train_samples_per_second': 20.473, 'train_steps_per_second': 0.639, 'total_flos': 1027721009645568.0, 'train_loss': 0.2612408071756363, 'epoch': 4.97})

In [ ]:
xlm_roberta_all_model.save_pretrained(f"./model_xlm_roberta_all")

In [ ]:
trainer_xlm_robeta_ar.train()

/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


wandb: Currently logged in as: jehadoumer. Use `wandb login --relogin` to force relogin


Step,Training Loss,Validation Loss,Macro F1,Accuracy,Precision,Recall
30,1.004800,0.794926,0.528926,0.676056,0.459802,0.643275
60,0.707200,0.418959,0.804980,0.830986,0.861485,0.815789
90,0.353300,0.177925,0.938596,0.943662,0.941147,0.938781
120,0.181400,0.134742,0.931437,0.936620,0.935213,0.932749
150,0.131700,0.121427,0.954337,0.957746,0.957011,0.954470


/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


TrainOutput(global_step=150, training_loss=0.4756895128885905, metrics={'train_runtime': 214.2833, 'train_samples_per_second': 22.05, 'train_steps_per_second': 0.7, 'total_flos': 310802724691200.0, 'train_loss': 0.4756895128885905, 'epoch': 5.0})

In [ ]:
xlm_roberta_ar_model.save_pretrained(f"./model_xlm_roberta_ar")

# Evaluation on Test Set

Loading best models and their trainers

In [ ]:
arabert_model = AutoModelForSequenceClassification.from_pretrained("./model_arabert")
#trainer_arabert = Trainer(model=arabert_model)


xlm_roberta_all_model = AutoModelForSequenceClassification.from_pretrained("./model_xlm_roberta_all")
#trainer_xlm_roberta_all = Trainer(model=xlm_roberta_all_model)

xlm_roberta_ar_model = AutoModelForSequenceClassification.from_pretrained("./model_xlm_roberta_ar")
#trainer_xlm_roberta_ar = Trainer(model=xlm_roberta_ar_model)


tokenizer_arabert = AutoTokenizer.from_pretrained("aubmindlab/bert-base-arabertv2")
tokenizer_xlm_roberta = AutoTokenizer.from_pretrained("xlm-roberta-base")


In [ ]:
from transformers import pipeline

classifier_arabert = pipeline("text-classification", model = arabert_model, tokenizer=tokenizer_arabert)

classifier_xlm_roberta_all = pipeline("text-classification", model = xlm_roberta_all_model, tokenizer=tokenizer_xlm_roberta)

classifier_xlm_roberta_ar = pipeline("text-classification", model = xlm_roberta_ar_model, tokenizer=tokenizer_xlm_roberta)

In [ ]:
label_map

{'fake': 0, 'real': 1, 'satire': 2}

In [ ]:
##the test sets are technically the same, (arabic test set)
#it is only because I have seperated them into different database objects somewhere above 
#and now it seems useless but have to deal with it

preds_arabert = classifier_arabert(test_ar_dataset.tweet)

preds_xlm_roberta_all = classifier_xlm_roberta_all(test_all_dataset.tweet)

preds_xlm_roberta_ar = classifier_xlm_roberta_ar(test_ar_dataset_xlm_roberta.tweet)


In [ ]:
def get_labels(preds):
  predictions  = []
  for pred in preds:
    if pred["label"] == 'LABEL_0':
      predictions.append("fake")
    elif pred["label"] == 'LABEL_1':
      predictions.append("real")
    elif pred["label"] == 'LABEL_2':
      predictions.append("satire")
  
  return predictions


In [ ]:
preds_arabert = get_labels(preds_arabert)
preds_xlm_roberta_all = get_labels(preds_xlm_roberta_all)
preds_xlm_roberta_ar = get_labels(preds_xlm_roberta_ar)

In [ ]:
from sklearn.metrics import classification_report, accuracy_score, precision_score, recall_score, f1_score

print(classification_report(preds_arabert, test_ar_dataset.label, digits=3))

              precision    recall  f1-score   support

        fake      0.988     0.976     0.982        83
        real      0.990     0.961     0.975       103
      satire      0.933     0.977     0.955        86

    accuracy                          0.971       272
   macro avg      0.970     0.971     0.971       272
weighted avg      0.971     0.971     0.971       272



In [ ]:
print(classification_report(preds_xlm_roberta_all, test_all_dataset.label, digits=3))

              precision    recall  f1-score   support

        fake      1.000     0.965     0.982        85
        real      0.990     0.952     0.971       104
      satire      0.911     0.988     0.948        83

    accuracy                          0.967       272
   macro avg      0.967     0.968     0.967       272
weighted avg      0.969     0.967     0.967       272



In [ ]:
print(classification_report(preds_xlm_roberta_ar, test_ar_dataset_xlm_roberta.label, digits=3))

              precision    recall  f1-score   support

        fake      1.000     0.976     0.988        84
        real      1.000     0.952     0.976       105
      satire      0.922     1.000     0.960        83

    accuracy                          0.974       272
   macro avg      0.974     0.976     0.974       272
weighted avg      0.976     0.974     0.975       272



|                          | F1    | Precision | Recall | Accuracy |
|--------------------------|-------|-----------|--------|----------|
| AraBERT with Ar          | 0.971 |   0.970   |  0.971 |   0.971  |
| XLM-RoBERTa with En & Ar | 0.967 |   0.967   |  0.968 |   0.967  |
| XLM-RoBERTa with Ar      | 0.974 |   0.974   |  0.976 |   0.974  |